In [147]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [148]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [149]:
!pip install sacrebleu
from datasets import load_dataset, load_metric
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [150]:
text = load_dataset("opus_euconst", "de-en")
metric = load_metric("sacrebleu")

  0%|          | 0/1 [00:00<?, ?it/s]

In [151]:
text

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 8772
    })
})

In [152]:
text = text["train"].train_test_split(test_size=0.2)

In [153]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "de"

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task

In [154]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "de"
prefix = "translate English to German: "

def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [155]:
tokenized_sentences = text.map(preprocess_function, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [156]:
tokenized_sentences['test']['translation'][:15]

[{'de': 'Ab dem Zeitpunkt, zu dem die Zahl der Mitglieder des Rates der Europäischen Zentralbank 21 übersteigt, hat jedes Mitglied des Direktoriums eine Stimme und beträgt die Zahl der stimmberechtigten Präsidenten der nationalen Zentralbanken 15.  ',
  'en': 'As from the date on which the number of members of the Governing Council exceeds 21, each member of the Executive Board shall have one vote and the number of governors with a voting right shall be 15.  '},
 {'de': 'Dieses Recht umfasst die Freiheit, die Religion oder Weltanschauung zu wechseln, und die Freiheit, seine Religion oder Weltanschauung einzeln oder gemeinsam mit anderen öffentlich oder privat durch Gottesdienst, Unterricht, Bräuche und Riten zu bekennen.  ',
  'en': 'This right includes freedom to change religion or belief and freedom, either alone or in community with others and in public or in private, to manifest religion or belief, in worship, teaching, practice and observance.  '},
 {'de': 'Sie schlägt ihnen die z

In [157]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [158]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

loading configuration file https://huggingface.co/t5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penal

In [159]:
model_base = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_pena

In [160]:
tokenizer_base = AutoTokenizer.from_pretrained("t5-base")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tas

In [161]:
from transformers import DataCollatorForSeq2Seq
# adding <pad> token
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [162]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [163]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sentences["train"],
    eval_dataset=tokenized_sentences["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
#trainer.train()

In [164]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [165]:
model_base.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [166]:
def translate(sentencee, tokenizer, model1):
  sentence_ = prefix + sentencee
  input_ids = tokenizer(sentence_, max_length=128, padding=True, truncation=True,
                        return_tensors='pt', add_special_tokens=True).input_ids
  out = model1.generate(input_ids, max_length=128)
  translated_sentence = tokenizer.decode(out[0], skip_special_tokens=True)
  return translated_sentence

In [167]:
# Translate example
check = "A decision requiring a qualified majority shall be adopted if the \
        votes cast in favour represent at least two thirds of the subscribed \
        capital of the European Central Bank and represent at least half of \
        the shareholders."

In [168]:
translate(check, tokenizer, model)

'Eine Entscheidung, die eine qualifizierte Mehrheit erfordert, wird angenommen, wenn die Ja-Stimmen mindestens zwei Drittel des unterzeichneten Kapitals der Europäischen Zentralbank ausmachen und mindestens die Hälfte der Aktionäre vertreten.'

In [169]:
translate(check, tokenizer_base, model_base)

'Eine Entscheidung mit qualifizierter Mehrheit wird angenommen, wenn die dafür abgegebenen Stimmen mindestens zwei Drittel des unterzeichneten Kapitals der Europäischen Zentralbank und mindestens die Hälfte der Aktionäre ausmachen.'

In [170]:
"""True version : Ein Beschluss, der einer qualifizierten Mehrheit bedarf, kommt
        zustande, wenn die abgegebenen Ja-Stimmen mindestens zwei Drittel der
        gezeichneten Kapital der Europäischen Zentralbank und repräsentieren
        mindestens die Hälfte von die Aktionäre."""

MODEL EVALUATING

In [171]:
from nltk.translate.bleu_score import sentence_bleu

In [172]:
def bleu_score(references, hypotesis, tokenizer, weights=(0.2, 0.2, 0.2, 0.2)):
  return sentence_bleu(references=[tokenizer(references, max_length=128, truncation=True).input_ids], 
                        hypothesis=tokenizer(hypotesis, max_length=128, truncation=True).input_ids, weights=weights)

In [173]:
bleu_summary_1 = 0
for b in tokenized_sentences['train']['translation'][:300]:
  eng = b['en']
  de = b['de']
  translated = translate(eng, tokenizer, model)
  bleu_summary_1 += bleu_score(de, translated, tokenizer)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [174]:
avg1 = bleu_summary_1 / 300 # avg for 300 sentences
avg1

0.42959479143562457

In [175]:
bleu_summary_2 = 0
for b in tokenized_sentences['train']['translation'][:300]:
  eng = b['en']
  de = b['de']
  translated = translate(eng, tokenizer_base, model_base)
  bleu_summary_2 += bleu_score(de, translated, tokenizer_base)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [176]:
avg2 = bleu_summary_2 / 300
avg2

0.44822018202089436

Pretrained models from t5 (t5-small and t5-base) show good performance on translating sentences from "opus_euconst" dataset || as shown in example thanslation is almost identical to true translation. bleu score can be improved by additional training, but it will take a lot of time , considering, that autor got bad GPU.